In [1]:
import pandas as pd
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
from deeppavlov.models.embedders.fasttext_embedder import FasttextEmbedder
import nltk
from deeppavlov import configs, build_model

[nltk_data] Downloading package punkt to /home/vsmaxim8/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vsmaxim8/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/vsmaxim8/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/vsmaxim8/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [2]:
import numpy as np

In [3]:
df = pd.read_csv("../datasets/lenta-ru-news.csv")
df[['title', 'text']] = df[['title', 'text']].apply(lambda i: i.replace('\xa0', ' '))
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)

In [4]:
tokens = df['title'].apply(lambda i: nltk.word_tokenize(i)).to_list()

In [5]:
len(df)

10110

In [6]:
para_model = FasttextEmbedder('/home/vsmaxim8/.deeppavlov/downloads/embeddings/ft_native_300_ru_wiki_lenta_lower_case.bin')

2020-05-14 09:26:50.155 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 53: [loading fastText embeddings from `/home/vsmaxim8/.deeppavlov/downloads/embeddings/ft_native_300_ru_wiki_lenta_lower_case.bin`]



In [7]:
def get_mean_embeddings(tokens):
    return [np.mean(sent_embeddings, axis=0) for sent_embeddings in para_model(tokens)]

In [8]:
def get_mean_sentence_embeddings_from_text(text):
    tokens = [nltk.word_tokenize(sentence) for sentence in nltk.sent_tokenize(text)]
    return get_mean_embeddings(tokens)

In [9]:
embeddings = para_model(tokens)

In [10]:
mean_embeddings = [np.mean(sent_embeddings, axis=0) for sent_embeddings in embeddings]

In [11]:
df['title_embeddings'] = mean_embeddings

In [ ]:
df['text_embeddings'] = df['text'].apply(lambda i: get_mean_sentence_embeddings_from_text(i))

In [ ]:
del para_model

In [ ]:
df.to_pickle('../datasets/lenta-ru-news-embedded.pkl')